# Fleet Analytics of a Logistics Company

Install packages

In [ ]:
pip install geopy

In [ ]:
pip install openrouteservice

In [ ]:
pip install folium

Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


### Parse the data (locations and routes)

In [ ]:
# Load locations
import pandas as pd

# Load the Excel file and use the first row as headers
file_path = 'data/Filial_Master_NWZZ.xlsx'  # Update the path as needed
locations = pd.read_excel(file_path, sheet_name=1, header=0)  # 'header=0' makes the first row the column headers

# Display the header (first few rows)
locations.head()


In [ ]:
# Load routes

file_path = 'data/StrategischerTourenplan_HSLU.xlsx'  
routes = pd.read_excel(file_path, sheet_name=0, header=0)  # 'header=0' makes the first row the column headers

# Display the header (first few rows)
routes.head()

### Simple analytics

In [ ]:
# Number of locations
num_locations = locations.shape[0]
print('Number of locations: ' + str(num_locations))

# Operating days
operating_days = routes['Wochentag'].unique().shape[0]
print('Operating days per week: ' + str(operating_days))

# Number of routes
num_routes = routes['Tournummer'].unique().shape[0]
print('Number of routes: ' + str(num_routes))

### Remove multiple entries of same VST

In [ ]:
# Filter multiple VSTs
routes_grouped = routes.groupby(('Tournummer')).first().__deepcopy__()
routes_grouped.head()


In [ ]:
# add location address to routes
routes_grouped.shape

In [ ]:
#in locations dataframe replace column name 'VST-Nummer' with 'SAP'
locations.rename(columns={'VST-Nummer':'SAP'}, inplace=True)
locations.head()

In [ ]:
# Find duplicates in the 'locations' dataframe based on the 'SAP' column
duplicate_locations = locations[locations.duplicated(subset='SAP', keep=False)]

# Display the duplicates
print(duplicate_locations)


In [ ]:
# Remove duplicates from the 'locations' dataframe based on the 'SAP' column
locations = locations.drop_duplicates(subset='SAP')

In [ ]:
# Find duplicates in the 'locations' dataframe based on the 'SAP' column
duplicate_locations = locations[locations.duplicated(subset='SAP', keep=False)]

# Display the duplicates
print(duplicate_locations)

In [ ]:
# Set 'SAP' as the index in both dataframes for alignment
routes_with_address = routes_grouped.join(locations.set_index('SAP'), on='SAP', how='left')

# This way, the original index of routes_grouped is preserved, and no extra rows are added.
routes_with_address.head()

In [ ]:
routes_with_address.to_excel('data/routes_with_address.xlsx')

In [ ]:
# In routes_with_address dataframe, make PLZ column integer
routes_with_address['PLZ'] = routes_with_address['PLZ'].astype(int)

In [ ]:
routes_with_address.shape


### Add coordinates to the locations


In [ ]:
# Geocoding with geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="geoapi")

# Create a rate limiter to handle many requests (1 request per second to avoid being blocked)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Function to get full address for geocoding
def get_full_address(row):
    return f"{row['Strasse']}, {row['PLZ']}, {row['Stadt']}"

# Limit the dataframe to the first 3rows
routes_with_address = routes_with_address.iloc[:20]

# Apply the geocode function to each row
routes_with_address['full_address'] = routes_with_address.apply(get_full_address, axis=1)

# Geocode each address and extract latitude and longitude
routes_with_address['location'] = routes_with_address['full_address'].apply(geocode)
routes_with_address['latitude'] = routes_with_address['location'].apply(lambda loc: loc.latitude if loc else None)
routes_with_address['longitude'] = routes_with_address['location'].apply(lambda loc: loc.longitude if loc else None)
routes_with_address.head()

In [ ]:
routes_with_address.to_excel('export/routes_with_address.xlsx')

### Display the locations on a map

In [ ]:
import folium

# Create a Folium map centered at the average latitude and longitude
map_center = [routes_with_address['latitude'].mean(), routes_with_address['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=10)

# Add markers to the map for each location
for _, row in routes_with_address.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['Strasse']}, {row['PLZ']}, {row['Stadt']}",
        tooltip=row['Stadt']
    ).add_to(mymap)

mymap


### Calculate the distance of routes

In [ ]:
import openrouteservice
from geopy.geocoders import Nominatim

# Initialize the OpenRouteService client with your API key
API_KEY = 'your-api-key'
client = openrouteservice.Client(key=API_KEY)

# Initialize Nominatim geocoder to convert addresses to coordinates
geolocator = Nominatim(user_agent="geoapiExercises")

# Function to convert address to coordinates
def get_coordinates(address):
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude)

# Define the fixed address
fixed_address = "xxx, yyyy, Switzerland"

# Get coordinates for the fixed address
geocode_fixed = client.pelias_search(fixed_address)
fixed_coords = geocode_fixed['features'][0]['geometry']['coordinates']
fixed_lat, fixed_lon = fixed_coords[1], fixed_coords[0]


# Calculate distances and add to dataframe
def calculate_distance(lat1, lon1, lat2, lon2):
    routes = client.directions(
        coordinates=[[lon1, lat1], [lon2, lat2]],
        profile='driving-car',
        format='geojson'
    )
    distance = routes['features'][0]['properties']['segments'][0]['distance']
    return distance / 1000  # Convert meters to kilometers

# Calculate distance from the fixed address for each row
routes_with_address['distance_from_fixed'] = routes_with_address.apply(
    lambda row: calculate_distance(fixed_lat, fixed_lon, row['latitude'], row['longitude']),
    axis=1
)
routes_with_address.head()

### Extract a route and show it on map

In [ ]:
import openrouteservice
import folium

# Initialize client with OpenRouteService API key
client = openrouteservice.Client(key='your-api-key')

# Define the fixed address and get its coordinates
fixed_address = "xxx, yyy, zzz"
geocode_fixed = client.pelias_search(fixed_address)
fixed_coords = geocode_fixed['features'][0]['geometry']['coordinates']
fixed_lat, fixed_lon = fixed_coords[1], fixed_coords[0]

# Define a sample address to route to
destination_row = routes_with_address.loc['1S1011']
destination_lat = destination_row['latitude']
destination_lon = destination_row['longitude']
destination_coords = [destination_lon, destination_lat]

# Get route data
routes = client.directions(
    coordinates=[[fixed_lon, fixed_lat], destination_coords],
    profile='driving-car',
    format='geojson'
)

# Extract route geometry
route_geom = routes['features'][0]['geometry']['coordinates']

# Create a folium map centered around the fixed address
m = folium.Map(location=[fixed_lat, fixed_lon], zoom_start=14)

# Add route to map
folium.PolyLine(
    locations=[[coord[1], coord[0]] for coord in route_geom],
    color='blue',
    weight=5,
    opacity=0.7
).add_to(m)

# Add markers for start and end points
folium.Marker(
    location=[fixed_lat, fixed_lon],
    popup='Start: Fixed Address',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[destination_coords[1], destination_coords[0]],
    popup='End: Destination Address',
    icon=folium.Icon(color='red')
).add_to(m)

m

# more info here: https://openrouteservice.org/disaster-optimization/
